In [23]:
# Import dependencies
import requests
import json
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
import time

# Load API Key
load_dotenv(find_dotenv('api.env'))
api_key = os.getenv("POLYGON_KEY")

base_url = "https://api.polygon.io"

stocksTicker = "SPY"
multiplier = 5
timespan = 'minute'

t_from = "2024-02-26"
t_to = "2024-03-22"

query_url = f"{base_url}/v2/aggs/ticker/{stocksTicker}/range/{multiplier}/{timespan}/{t_from}/{t_to}?adjusted=true&sort=asc&apiKey={api_key}"

In [24]:
# Get the response
response = requests.get(query_url)
response

# Retrieve reviews
reviews = response.json()

# Print results in JSON format
print(json.dumps(reviews, indent=4))

{
    "ticker": "SPY",
    "queryCount": 5000,
    "resultsCount": 1212,
    "adjusted": true,
    "results": [
        {
            "v": 4310,
            "vw": 507.3061,
            "o": 507.32,
            "c": 507.35,
            "h": 507.4,
            "l": 507.22,
            "t": 1708938000000,
            "n": 91
        },
        {
            "v": 911,
            "vw": 507.1577,
            "o": 507.29,
            "c": 507.06,
            "h": 507.29,
            "l": 507.06,
            "t": 1708938300000,
            "n": 20
        },
        {
            "v": 4157,
            "vw": 507.1101,
            "o": 507.15,
            "c": 507.1,
            "h": 507.16,
            "l": 507.08,
            "t": 1708938600000,
            "n": 23
        },
        {
            "v": 100,
            "vw": 507.25,
            "o": 507.25,
            "c": 507.25,
            "h": 507.25,
            "l": 507.25,
            "t": 1708938900000,
            "n": 1
        },

In [31]:
reviews_df = pd.json_normalize(reviews["results"])
reviews_df = reviews_df.rename(columns={'v':'Volume', 'vw':'Volume Weighted', 'o':'Open Price', 'c':'Close Price', 'h':'Highest Price', 'l':'Lowest Price', 't':'Timestamp', 'n':'Number of Transaction'})
reviews_df['Timestamp'] = pd.to_datetime(reviews_df['Timestamp'], unit='ms')
reviews_df = reviews_df.set_index('Timestamp')
reviews_df

,Volume,Volume Weighted,Open Price,Close Price,Highest Price,Lowest Price,Number of Transaction
Timestamp,,,,,,,
2024-02-26 09:00:00,4310.0,507.3061,507.32,507.350,507.4000,507.220,91
2024-02-26 09:05:00,911.0,507.1577,507.29,507.060,507.2900,507.060,20
2024-02-26 09:10:00,4157.0,507.1101,507.15,507.100,507.1600,507.080,23
2024-02-26 09:15:00,100.0,507.2500,507.25,507.250,507.2500,507.250,1
2024-02-26 09:20:00,101.0,507.3200,507.32,507.320,507.3200,507.320,2
...,...,...,...,...,...,...,...
2024-03-05 15:45:00,1252980.0,509.7186,507.89,507.530,508.2800,507.520,8085
2024-03-05 15:50:00,555425.0,507.7817,507.53,507.795,508.0399,507.500,6406
2024-03-05 15:55:00,540502.0,507.9674,507.81,508.120,508.1250,507.795,6026
